In [31]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
import numpy as np
import pandas as pd
import ntpath
import glob
import deepconfig

#from keras.utils.visualize_util import plot
#from IPython.display import Image, display

from CMS_SURF_2016.utils.preprocessing import *
from CMS_SURF_2016.utils.callbacks import OverfitStopping, SmartCheckpoint
from CMS_SURF_2016.utils.batch import batchAssertArchived, batchExecuteAndTestTrials
from CMS_SURF_2016.utils.archiving import *
from CMS_SURF_2016.utils.analysistools import findsubsets
from CMS_SURF_2016.layers.lorentz import Lorentz, _lorentz
from CMS_SURF_2016.layers.slice import Slice

from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, merge, Input, Flatten, Lambda, LSTM, Masking
from keras.engine.topology import Layer
from keras.callbacks import EarlyStopping
from keras.utils.visualize_util import plot
from keras.layers.advanced_activations import LeakyReLU


dc = deepconfig.deepconfig(gpu='gpu0', backend='theano')




using gpu0
using theano


In [32]:
#The observables taken from the table
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'PT_ET','Eta', 'Phi', 'Charge', 'X', 'Y', 'Z',\
                     'Dxy', 'Ehad', 'Eem', 'MuIso', 'EleIso', 'ChHadIso','NeuHadIso','GammaIso', "ObjType"]
vecsize = len(observ_types)
epochs = 30
batch_size = 100

label_dir_pairs = \
            [   ("ttbar", "/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/"),
                ("wjet", "/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/"),
                ("qcd", "/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/")
            ]


In [33]:
#Find all the subsets of label_dir_pairs and store them as sorted lists
ldpsubsets = []
#ldpsubsets = [sorted(list(s)) for s in findsubsets(label_dir_pairs)]
#Make sure that we do 3-way classification as well
ldpsubsets.append(label_dir_pairs)
#ldpsubsets=label_dir_pairs


In [ ]:
def genModel(name,out_dim, depth, lstm_activation="relu", lstm_dropout = 0.0, dropout=0.0,output_activation="sigmoid"):
    inputs = []
    mergelist = []
    for i, profile in enumerate(object_profiles):
        inp = a = Input(shape=(profile.max_size + 1*(profile.punctuation != None), vecsize), name="input_"+str(i))
        inputs.append(inp)
        mergelist.append(a)
    a = merge(mergelist,mode='concat',concat_axis=1, name="merge")
    for i in range(depth):
        a = Masking(mask_value=0.0)(a)
        a = LSTM(vecsize,
                 input_shape=(None,vecsize),
                 dropout_W=lstm_dropout,
                 dropout_U=lstm_dropout,
                 activation=lstm_activation,
                 name = "lstm_" +str(i))(a)
        if(dropout > 0.0):
            a =  Dropout(dropout, name="dropout_"+str(i))(a)
    dense_out = Dense(out_dim, activation=output_activation, name='main_output')(a)
    model = Model(input=inputs, output=dense_out, name=name)
    return model

In [ ]:
archive_dir = "/data/shared/Delphes/keras_archive/"
patience = 8
earlyStopping = EarlyStopping(verbose=1, patience=patience)
trial_tups = []
#Loop over all subsets
print(ldpsubsets)
for ldp in ldpsubsets:
    labels = [x[0] for x in ldp]
    #for sort_on in ["PT_ET", "Phi", "Eta"]:
    for sort_on in ["Phi"]:
        #Use object maxes from Find_Maxes_From Query
        object_profiles = [ObjectProfile("Electron",-1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":1}),
                            ObjectProfile("MuonTight", -1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":2}),
                            ObjectProfile("Photon", -1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":3}),
                            ObjectProfile("MissingET", 1, addColumns={"ObjType":4}),
                            ObjectProfile("EFlowPhoton",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":5}), 
                            ObjectProfile("EFlowNeutralHadron",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":6}), 
                            ObjectProfile("EFlowTrack",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":7})]  

        resolveProfileMaxes(object_profiles, ldp)

        dps, l = getGensDefaultFormat(archive_dir, (75000,20000,20000), 135000, \
                             object_profiles,ldp,observ_types,batch_size=batch_size,megabytes=100, verbose=0)

        dependencies = batchAssertArchived(dps)
        train, num_train = l[0]
        val,   num_val   = l[1]
        test,  num_test  = l[2]
        max_q_size = l[3]
        print("MAXQ: ",max_q_size)

       
        for name in ['LSTM']:
            for depth in [1]:
                    for activation in ['tanh']:
                        for lstm_dropout in [0.0]:
                            for dropout in [0.0]:
                                activation_name = activation if isinstance(activation, str) \
                                                    else activation.__name__

                                model = genModel(name, len(labels), depth, activation, lstm_dropout, dropout,output_activation="softmax")

                                trial = KerasTrial(archive_dir, name=name, model=model)

                                trial.setTrain(train_procedure=train,
                                               samples_per_epoch=num_train
                                              )
                                trial.setValidation(val_procedure=val,
                                                   nb_val_samples=num_val)
                                
                                trial.setCompilation(loss='categorical_crossentropy',
                                          optimizer='rmsprop',
                                          metrics=['accuracy']
                                              )

                                trial.setFit_Generator( 
                                                nb_epoch=epochs,
                                                callbacks=[earlyStopping],
                                                max_q_size = max_q_size)
                                trial.write()


#                                print("EXECUTE: ", name,labels, depth, activation_name)
#                                trial.execute(custom_objects={"Lorentz":Lorentz,"Slice": Slice},
#                                             train_arg_decode_func=label_dir_pairs_args_decoder,
#                                             val_arg_decode_func=label_dir_pairs_args_decoder)


#                                trial.test(test_proc=test,
#                                             test_samples=num_test,
#                                             custom_objects={"Lorentz":Lorentz,"Slice": Slice},
#                                            arg_decode_func = label_dir_pairs_args_decoder)

                                trial_tups.append((trial, test, num_test, dependencies))

                                trial.to_record({"labels": labels,
                                                 "depth": depth,
                                                 "sort_on" : sort_on,
                                                 "activation": activation_name,
                                                 "dropout":dropout,
                                                 "lstm_dropout":lstm_dropout,
                                                 "query": None,
                                                 "patience" : patience,
                                                 "useObjTypeColumn" : True,
                                                 "output_activation" : "softmax"
                                                })
for tup in trial_tups:
    tup[0].summary()
batchExecuteAndTestTrials(trial_tups)
                
        

    

[[('ttbar', '/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/'), ('wjet', '/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/'), ('qcd', '/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/')]]
('MAXQ: ', 31.0)
--------------------------------------------------
TRIAL SUMMARY (ed30d9d9239c67bdcb2dcd3906149d6ab15feabf)
    Record_Info:
        name = [u'LSTM']
        
        
        activation = tanh, depth = 1, dropout = 0.0, labels = [u'ttbar', u'wjet', u'qcd'], lstm_dropout = 0.0, output_activation = softmax, patience = 8, query = None, sort_on = Phi, useObjTypeColumn = True
--------------------------------------------------
EXECUTE 'ed30d9d9239c67bdcb2dcd3906149d6ab15feabf'
Failed to load history at /data/shared/Delphes/keras_archive/blobs/ed30d/9d9239c67bdcb2dcd3906149d6ab15feabf/history.json

WARNING (theano.gof.cmodule): Removing key file /home/dweitekamp/.theano/compiledir_Linux-3.19--generic-x86_64-with-Ubuntu-15.04-vivid-x86_64-2.7.9-64/tmpTDtSd1/key.pkl because the corresponding module is gone from the file system.
WARNING (theano.gof.cmodule): Removing key file /home/dweitekamp/.theano/compiledir_Linux-3.19--generic-x86_64-with-Ubuntu-15.04-vivid-x86_64-2.7.9-64/tmpcUYOMn/key.pkl because the corresponding module is gone from the file system.
WARNING (theano.gof.cmodule): Removing key file /home/dweitekamp/.theano/compiledir_Linux-3.19--generic-x86_64-with-Ubuntu-15.04-vivid-x86_64-2.7.9-64/tmpNTM7Ae/key.pkl because the corresponding module is gone from the file system.
WARNING (theano.gof.cmodule): Removing key file /home/dweitekamp/.theano/compiledir_Linux-3.19--generic-x86_64-with-Ubuntu-15.04-vivid-x86_64-2.7.9-64/tmpnNhE7L/key.pkl because the corresponding module is gone from the file system.
WARNING (theano.gof.cmodule): Removing key file /home/dweitekamp/.theano


Failed to load weights at /data/shared/Delphes/keras_archive/blobs/ed30d/9d9239c67bdcb2dcd3906149d6ab15feabf/weights.h5
Epoch 1/30
225000/225000 [==============================] - 629s - loss: 0.9845 - acc: 0.4785 - val_loss: 0.8111 - val_acc: 0.5861
Epoch 2/30
225000/225000 [==============================] - 553s - loss: 0.7299 - acc: 0.6260 - val_loss: 0.6631 - val_acc: 0.6576
Epoch 3/30
225000/225000 [==============================] - 534s - loss: 0.6082 - acc: 0.6750 - val_loss: 0.5741 - val_acc: 0.6917
Epoch 4/30
225000/225000 [==============================] - 538s - loss: 0.5569 - acc: 0.6996 - val_loss: 0.5471 - val_acc: 0.7072
Epoch 5/30
225000/225000 [==============================] - 527s - loss: 0.5298 - acc: 0.7227 - val_loss: 0.5183 - val_acc: 0.7410
Epoch 6/30
225000/225000 [==============================] - 533s - loss: 0.4985 - acc: 0.7563 - val_loss: 0.5052 - val_acc: 0.7617
Epoch 7/30
225000/225000 [==============================] - 526s - loss: 0.4777 - acc: 0.7736

In [ ]:
Algo("sgd")